# Ascension Analytics

### Preamble

In [57]:
from __future__ import division
# import qgrid
import pprint
import simplejson
import pandas as pd

from firebase import firebase
from collections import defaultdict

from plotly.offline import init_notebook_mode, iplot
import cufflinks as cf

init_notebook_mode()
cf.go_offline()

In [58]:
url = 'https://ascension.firebaseio.com'
firebase = firebase.FirebaseApplication(url, None)
base = firebase.get('/', None)

# Cache Game Data
with open('/tmp/ascension.json','w') as json_file:
    simplejson.dump(base, json_file)

#### Load Cached Data

In [59]:
with open('/tmp/ascension.json') as json_file:
    base = simplejson.load(json_file)

#### Config

In [60]:
# Jupyter Notebook
from IPython.display import HTML
HTML('''<style> #notebook-container{width:1270px}
.reveal .slides {
    width: 100% !important;
}
</style>''')
HTML('''<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>''')

HTML('''<script>
code_show=true; 
function code_toggle() {
Reveal.initialize({
    width: '1350',
    height: '100%'
})
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Cufflinks
# cf.set_config_file(offline_show_link=False,theme='solar')
# cf.get_config_file()

#### Styles

In [61]:
# AVAILABLE THEMES

# for theme in cf.getThemes():
#     print theme
#     cf.datagen.lines().iplot(kind='bar', colorscale='set3', theme=theme)

# AVAILABLE SCALES
# cf.colors.scales()

COLORSCALE = 'rdylbu'
COLORSCALE_D = 'spectral'
COLORSCALE_C = 'ylgn'

#### Documentation

In [62]:
# Cufflinks Iplot
# help(pd.DataFrame.iplot)

### Utility Functions

In [63]:
# Set this pandas option to prevent the grid from being too big
pd.set_option('display.max_rows', 20)
# grid = lambda df: qgrid.show_grid(df)


import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

### Globals

In [64]:
LEAGUE = 'essos'
EPISODE = 60

### Data Structure

In [65]:
# base.keys()

In [66]:
# Episodes
episodes = pd.DataFrame(base['episodes'].values())

# Houses 
houses = pd.DataFrame(base['houses'].values())

# Characters
characters = pd.DataFrame(base['characters'].values())

# Players
players = pd.DataFrame(base['players'].values())[['alias','alias_short','facebook','first_name','full_name']]
players['id'] = 'facebook:' + players.facebook
players = players.set_index('id')
roster = pd.DataFrame(base['rosters'].values())

# Leaderboard
leaderboard = pd.concat([pd.DataFrame(y['scores'].values(), index=y['scores'].keys(), columns=[y['episode']]) for x,y in base['leaderboard'].iteritems() if LEAGUE in x], axis=1).sort_index(axis=1).sort_values(EPISODE, ascending=False)
try:
    leaderboard.index = leaderboard.index.to_series().apply(lambda id: players.ix[id,'first_name'])
except KeyError:
    print 'Leaderboard : Mapping only once'

# Episode Scores
index = [y['episode'] for x, y in base['episode_scores'].iteritems() if y['league'] == LEAGUE]
episode_scores = pd.DataFrame([y['scores'] for x, y in base['episode_scores'].iteritems() if y['league'] == LEAGUE], index=index).sort_index()
episode_scores

# Votes
votes = pd.DataFrame([y for x,y in base['votes'].iteritems() if y['league'] == LEAGUE])
try:
    votes.player = votes.player.apply(lambda id: players.ix[id,'first_name'])
except KeyError:
    print 'Votes : Mapping only once'
votes.set_index(['player','episode']).ix[:,votes.set_index(['player','episode']).columns.str.contains('vote')].sort_index()
    
# Episode Votes
episode_vote = [pd.DataFrame(y['votes']['values'], columns=y['votes']['houses'], index=y['votes']['characters']) for x,y in base['episode_votes'].iteritems() if y['league'] == LEAGUE]   

# Player Award Scores
player_award_scores = pd.DataFrame([flatten(y) for x,y in base['player_award_scores'].iteritems()if y['league'] == LEAGUE] )
try:
    player_award_scores.player = player_award_scores.player.apply(lambda id: players.ix[id,'first_name'])
except KeyError:
    print 'Player Votes : Mapping only once'

# Character Health
character_health = pd.DataFrame([flatten(y) for x,y in base['character_health'].iteritems() if LEAGUE in x]).groupby(['house','episode']).sum()

missions = pd.DataFrame([flatten(y) for x,y in base['missions'].iteritems() if LEAGUE == y['league']])
missions.ix[missions.assassination_target_house == '','assassination_agent assassination_target_character assassination_target_house'.split(' ')] = 'None'
missions.ix[missions.diplomatic_agent == '','diplomatic_agent diplomatic_target_house'.split(' ')] = 'None'
try:
    missions.player = missions.player.apply(lambda id: players.ix[id,'first_name'])
except KeyError:
    print 'Missions : Mapping only once'

# Player Chronicle - NEEDS FLATTENING
# player_chronicles = base['player_chronicles']

# Player Intelligence - NEEDS FLATTENING
player_intelligence = pd.DataFrame(base['player_intelligence'])

In [67]:
index = [flatten(y)['house'] for x,y in base['character_health'].iteritems() if LEAGUE in x and y['episode'] == EPISODE]
game_rosters = pd.DataFrame([flatten(y) for x,y in base['character_health'].iteritems() if LEAGUE in x and y['episode'] == EPISODE], index=index)
game_rosters.columns = game_rosters.columns.str.replace('health_','')
game_rosters = game_rosters.ix[:,1:-2]
game_rosters = game_rosters.T

In [68]:
player_houses_map = defaultdict(dict)

for player, vals in pd.DataFrame(base['players'].values())[['first_name','house']].iterrows():
    name = vals['first_name']
    for l,h in vals['house'].iteritems():
        player_houses_map[l][name] = h

In [69]:
player_awarded_points = player_award_scores.reset_index().groupby('player').sum()
player_awarded_points.ix[:,player_awarded_points.columns.str.contains('scores')]
player_awarded_points.columns = player_awarded_points.columns.str.replace('scores_','')

In [70]:
# game_rosters.T.ix[game_rosters.T.greyjoy.notnull(),'greyjoy']

#### Global

In [71]:
# Missions
missions_global = pd.DataFrame([flatten(y) for x,y in base['missions'].iteritems()])
missions_global.ix[missions_global.assassination_target_house == '','assassination_agent assassination_target_character assassination_target_house'.split(' ')] = 'None'
missions_global.ix[missions_global.diplomatic_agent == '','diplomatic_agent diplomatic_target_house'.split(' ')] = 'None'
try:
    missions_global.player = missions_global.player.apply(lambda id: players.ix[id,'first_name'])
except KeyError:
    print 'Missions : Mapping only once'
    
# Votes
votes_global = pd.DataFrame([y for x,y in base['votes'].iteritems()])
try:
    votes_global.player = votes_global.player.apply(lambda id: players.ix[id,'first_name'])
except KeyError:
    print 'Votes : Mapping only once'
votes_global = votes_global.set_index(['player','episode']).ix[:,votes_global.set_index(['player','episode']).columns.str.contains('vote')].sort_index()

# Data Graphics

In [72]:
try:
    10 - missions_global.groupby('player').count()['episode'].sort_values(ascending=False).iplot(kind='bar', title='Votes cast per Player in the Game');
except TypeError:
    pass

In [73]:
try:
    10 - missions.groupby('player').count()['episode'].sort_values(ascending=False).iplot(kind='bar', title='Votes cast per Player in {}'.format(LEAGUE.title()))
except TypeError:
    pass

## Characters

In [74]:
# Which Character got the most votes
pd.DataFrame(votes.ix[:,votes.columns.str.contains('vote')].unstack().value_counts()[:40]).iplot(
    kind='bar',
    title='Top Voted Character in {}'.format(LEAGUE.title()),
    margin=(30,30,120,60)
)

In [75]:
# Which Character got the most votes
pd.DataFrame(votes.ix[:,votes.columns.str.contains('vote')].unstack().value_counts()[40:]).iplot(
    kind='bar',
    title='Bottom Voted Character in {}'.format(LEAGUE.title()),
    margin=(30,30,120,60)
)

In [76]:
missing_votes = pd.DataFrame(list(set(characters.id).difference(votes.ix[:,votes.columns.str.contains('vote')].unstack().value_counts().index)))
missing_votes['sum'] = -1
missing_votes = missing_votes.set_index(0)
missing_votes.iplot(kind='bar',
    title='There\'s always worse.. you could always be a non-voted character')

In [77]:
import numpy as np

comp_votes = pd.merge(pd.DataFrame(votes.ix[:,votes.columns.str.contains('vote')].stack().value_counts(),columns=[LEAGUE]),
    pd.DataFrame(votes_global.ix[:,votes_global.columns.str.contains('vote')].stack().value_counts(),columns=['All']), how='outer',left_index=True,right_index=True).fillna(0).astype(int)

baseline = comp_votes[LEAGUE] / (comp_votes.All - comp_votes[LEAGUE])
portion = comp_votes[LEAGUE].sum() / (comp_votes.All.sum() - comp_votes[LEAGUE].sum())
baseline = baseline / portion
baseline[baseline == np.inf] = 10
ratio = pd.DataFrame(baseline,columns=['ratio']).sort_values('ratio', ascending=False)

In [78]:
ratio[ratio.ratio > 1].iplot(kind='bar',
                             color='green',
                            title='Overrepresented Characters in {} Votes'.format(LEAGUE.title()),
                            margin=(30,30,120,60))

In [79]:
ratio[ratio.ratio <= 1].iplot(kind='bar',
                             color='red',
                            title='Underrepresented Characters in {} Votes'.format(LEAGUE.title()),
                             margin=(30,30,120,60))

In [80]:
# Which Character got the most votes
pd.DataFrame(votes_global.ix[:,votes_global.columns.str.contains('vote')].stack().value_counts()[:40]).iplot(
    kind='bar',
    title='Top Voted Character across all Leagues',
    margin=(30,30,120,60)
)

In [81]:
points = defaultdict(int)
for col in votes.ix[:,votes.columns.str.contains('vote')]:
    if '1' in col:
        for char in votes[col]:
            points[char] += 20
    if '2' in col:
        for char in votes[col]:
            points[char] += 8
scored_points = pd.DataFrame(dict(points).values(), index=dict(points).keys(), columns=['points']).sort_values('points',ascending=False)
char_lookup = characters.set_index('id')

scored_points = scored_points.join(char_lookup)
scored_points['adjusted_points'] = scored_points.points * (6 - scored_points.prominence)
scored_points = scored_points.sort_values('adjusted_points',ascending=False)

In [82]:
# Which Character got the most votes
pd.DataFrame(scored_points.ix[:,'adjusted_points'][:40]).iplot(
    kind='bar',
    title='Top Scoring Characters in {}'.format(LEAGUE.title()),
    margin=(30,30,120,60)
)

In [83]:
pd.DataFrame(scored_points.ix[:,'adjusted_points'][40:]).iplot(
    kind='bar',
    title='Bottom Scoring Characters in {}'.format(LEAGUE.title()),
    margin=(30,30,120,60)
)

In [84]:
scored_points.groupby('prominence').sum()[['points','adjusted_points']].iplot(
    kind='bar',
    title='Total Points Awarded per Prominence Level in {}'.format(LEAGUE),
    xTitle='Prominence Level')

In [85]:
scored_points.groupby('diplomacy').sum()[['points','adjusted_points']].iplot(
    kind='bar',
    title='Total Points Awarded per Diplomatic Level in {}'.format(LEAGUE),
    xTitle='Diplomacy Level')

In [86]:
scored_points.groupby('violence').sum()[['points','adjusted_points']].iplot(
    kind='bar',
    title='Total Points Awarded per Violence Level in {}'.format(LEAGUE),
    xTitle='Violence Level')

In [87]:
auto_votes = defaultdict(int)
for player_name, idx in votes.groupby('player'):
    h = player_houses_map[LEAGUE][player_name]
#     print game_rosters[h][game_rosters[h].notnull()].keys().tolist()
    r = game_rosters[h][game_rosters[h].notnull()].keys().tolist()
    for char, count in idx.ix[:,votes.columns.str.contains('vote')].unstack().value_counts()[:40].iteritems():
        if char in r:
            auto_votes[player_name] += count
#     auto_votes[h]
# pd.DataFrame(votes)
pd.DataFrame(dict(auto_votes).values(), index=dict(auto_votes).keys(),columns=['votes']).sort_values('votes',ascending=False).iplot(
    kind='bar',
    title='Votes casted for own Characters in {}'.format(LEAGUE.title())
)

In [88]:
awards = {}
for award in ['damage','wit','jockey','support','style']:
    award_votes = votes.ix[:, votes.columns.str.contains(award)]
    
    award_points = defaultdict(int)

    for col in award_votes:
        if '1' in col:
            for char in votes[col]:
                award_points[char] += 20
        if '2' in col:
            for char in votes[col]:
                award_points[char] += 8

    scored_points = pd.DataFrame(dict(award_points).values(), index=dict(award_points).keys(), columns=['points']).sort_values('points',ascending=False)
    char_lookup = characters.set_index('id')

    scored_points = scored_points.join(char_lookup)
    scored_points['adjusted_points'] = scored_points.points * (6 - scored_points.prominence)
    scored_points = scored_points.sort_values('adjusted_points',ascending=False)
    awards[award] = scored_points['adjusted_points']

In [89]:
award_code = 'wit'
awards[award_code].ix[:25].iplot(
    kind='bar',
    title='Awarded Points in the {} category in {}'.format(award_code.title(),LEAGUE.title()),
    margin=(50,50,120,60)
)

In [90]:
award_code = 'damage'
awards[award_code].ix[:25].iplot(
    kind='bar',
    title='Awarded Points in the {} category in {}'.format(award_code.title(),LEAGUE.title()),
    margin=(50,50,120,60)
)

In [91]:
award_code = 'jockey'
awards[award_code].ix[:25].iplot(
    kind='bar',
    title='Awarded Points in the {} category in {}'.format(award_code.title(),LEAGUE.title()),
    margin=(50,50,120,60)
)

In [92]:
award_code = 'style'
awards[award_code].ix[:25].iplot(
    kind='bar',
    title='Awarded Points in the {} category in {}'.format(award_code.title(),LEAGUE.title()),
    margin=(50,50,120,60)
)

In [93]:
award_code = 'support'
awards[award_code].ix[:25].iplot(
    kind='bar',
    title='Awarded Points in the {} category in {}'.format(award_code.title(),LEAGUE.title()),
    margin=(50,50,120,60)
)

In [94]:
award_code = 'wit'
player_awarded_points[[award_code]].sort_values(award_code,ascending=False).iplot(
    kind='bar',
    title='Winner of the {} award in {}'.format(award_code.title(),LEAGUE.title()),
    margin=(50,50,120,60)
)

In [95]:
award_code = 'damage'
player_awarded_points[[award_code]].sort_values(award_code,ascending=False).iplot(
    kind='bar',
    title='Winner of the {} award in {}'.format(award_code.title(),LEAGUE.title()),
    margin=(50,50,120,60)
)

In [96]:
award_code = 'jockey'
player_awarded_points[[award_code]].sort_values(award_code,ascending=False).iplot(
    kind='bar',
    title='Winner of the {} award in {}'.format(award_code.title(),LEAGUE.title()),
    margin=(50,50,120,60)
)

In [97]:
award_code = 'style'
player_awarded_points[[award_code]].sort_values(award_code,ascending=False).iplot(
    kind='bar',
    title='Winner of the {} award in {}'.format(award_code.title(),LEAGUE.title()),
    margin=(50,50,120,60)
)

In [98]:
award_code = 'support'
player_awarded_points[[award_code]].sort_values(award_code,ascending=False).iplot(
    kind='bar',
    title='Winner of the {} award in {}'.format(award_code.title(),LEAGUE.title()),
    margin=(50,50,120,60)
)

In [99]:
player_awarded_points = player_award_scores.reset_index().groupby('player').sum()
player_awarded_points.ix[:,player_awarded_points.columns.str.contains('scores')]
player_awarded_points.columns = player_awarded_points.columns.str.replace('scores_','')
player_awarded_points.ix[:,1:].iplot(
    kind='bar',
#     barmode='stack',
    title='Point Distribution per Award per Player in {}'.format(LEAGUE.title()))


# player_award_scores.ix[:,player_award_scores.columns.str.contains('scores')]
# player_award_scores.reset_index().groupby('player')


## Houses

In [100]:
scored_points.groupby('house').sum()[['points','adjusted_points']].iplot(
    kind='bar',
    title='Total Points Awarded per House Characters in {}'.format(LEAGUE),
    xTitle='Character House')

## Missions

In [101]:
missions.groupby('diplomatic_target_house').count().ix[1:,'player'].iplot(kind='bar', title='Diplomatic Missions against a House in {}'.format(LEAGUE.title()))

In [102]:
missions.groupby('assassination_target_house').count().ix[1:,'player'].iplot(kind='bar', title='Assassination Missions against a House in {}'.format(LEAGUE.title()))

In [103]:
missions.groupby(['player','diplomatic_target_house']).count()['episode'].unstack().fillna(0).iplot(
            kind='heatmap',
            colorscale='ylgnbu',
            title='Diplomatic Target Houses by Player in {}'.format(LEAGUE.title()),
            xTitle='Players',
            margin=(100,100,80,80))

In [104]:
missions.groupby(['player','diplomatic_target_house']).count()['episode'].unstack().fillna(0)['None'].iplot(kind='bar', title='Rounds Diplomatic Agents spent on the Bench in {}'.format(LEAGUE.title()))

In [105]:
missions.groupby(['player','assassination_target_house']).count()['episode'].unstack().fillna(0).iplot(
            kind='heatmap',
            colorscale='ylgnbu',
            title='Assassination Target Houses by Player in {}'.format(LEAGUE.title()),
            xTitle='Players')

In [106]:
missions.groupby(['player','assassination_target_house']).count()['episode'].unstack().fillna(0)['None'].iplot(kind='bar', title='Rounds Assassins spent on the Bench in {}'.format(LEAGUE.title()))

In [107]:
game_rosters.T.sum().iplot(
    kind='bar',
    title='Remaining Health on Roster in {}'.format(LEAGUE.title()))

In [108]:
# TODO : How many times did BOLTON ability save him?!

### Leaderboard

# And the Winner is....

In [109]:
leaderboard.T.iplot(kind='line',
            colorscale='ylorrd',
            title='Cummulative Score per Player in {}'.format(LEAGUE.title()),
            xTitle='Episode')

In [110]:
leaderboard.sort_values(EPISODE).T.iplot(kind='heatmap',
            colorscale='ylorrd',
            title='Cummulative Score per Player in {}'.format(LEAGUE.title()),
            xTitle='Episode')

In [111]:
import scipy.stats as ss
leaderboard.apply(lambda v : len(v) - ss.rankdata(v) + 1).astype(int).T.iplot(kind='line',
            colorscale='ylorrd',
            title='Player Rank per Episode in {}'.format(LEAGUE.title()),
            xTitle='Episode',
            yTitle='Rank',
            layout_update={"yaxis" : {"autorange" : "reversed",'color':'#EEE'}}                                                                
)

# ... till next year!